In [2]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
import os
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world, construct_view_matrix
from visualization import load_depth
import progressbar
from pointcloud_to_voxelmap import pointclouds_to_voxelmap
from joblib import Parallel, delayed
from configparser import ConfigParser
from PIL import Image
import pickle

In [3]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file

conn = visualization.get_connection()
cur = conn.cursor()

CONFIG = ConfigParser()
CONFIG.read(ini_file)
in_directory = CONFIG["Images"]["Tiff"]
out_directory = CONFIG["Images"]["MlDatasetVoxel"]


In [4]:
cur.execute("""SELECT DISTINCT scene_id \
  FROM snapshots \
  WHERE run_id = 6 \
  """)

scenes = []
for row in cur:
    res = dict(row)
    scenes.append(res)

print('There are {} scenes'.format(len(scenes)))


There are 8438 scenes


### functions for each scene

In [7]:
def get_base_name(name):
    return os.path.basename(os.path.splitext(name)[0])


def scene_to_pointcloud(cameras):
    pointclouds = []
    cam_positions = []
    
    for cam in cameras:
        pointcloud = camera_to_pointcloud(cam)
        pointclouds.append(pointcloud)
        cam_positions.append(cam['camera_pos'])
    return pointclouds, cam_positions


def get_main_image_name(cameras):
    for cam in cameras:
        if cam['camera_relative_rotation'] == np.array([0, 0, 0]):
            return cam['imagepath']
    raise Exception('no main image')

    
def load_scene_db_data(scene_id):
    conn = visualization.get_connection()
    cur = conn.cursor()
    cur.execute("""SELECT snapshot_id, imagepath, cam_near_clip, camera_fov, width, height, \
      ARRAY[st_x(camera_relative_rotation), st_y(camera_relative_rotation), st_z(camera_relative_rotation)] as camera_relative_rotation, \
      ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as camera_pos, \
      ARRAY[st_x(camera_rot), st_y(camera_rot), st_z(camera_rot)] as camera_rot \
      FROM snapshots \
      WHERE scene_id = '{}'
      ORDER BY timestamp ASC \
    """.format(scene_id))

    cameras = []
    for row in cur:
        res = dict(row)
        res['camera_rot'] = np.array(res['camera_rot'])
        res['camera_pos'] = np.array(res['camera_pos'])
        res['camera_relative_rotation'] = np.array(res['camera_relative_rotation'])
        res['view_matrix'] = construct_view_matrix(res['camera_pos'], res['camera_rot'])
        res['proj_matrix'] = construct_proj_matrix(res['height'], res['width'], res['camera_fov'], res['cam_near_clip'])
        cameras.append(res)
    return cameras


def camera_to_pointcloud(cam):
    name = cam['imagepath']
    depth = load_depth(name)
    vecs = points_to_homo(res, depth)
    vecs_p = ndc_to_view(vecs, res['proj_matrix'])
    vecs_p_world = view_to_world(vecs_p, res['view_matrix'])
    return vecs_p_world


def scene_to_voxelmap(scene_id):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    pointclouds, cam_positions = scene_to_pointcloud(scene_id)
    voxelmap = pointclouds_to_voxelmap(pointclouds, cam_positions)
    return voxelmap


def convert_tiff(in_directory, out_directory, out_name, name):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    out_format = 'jpg'
    outfile = os.path.join(out_directory, "{}.{}".format(out_name, out_format))
    if os.path.exists(outfile):
        return

    try:
        infile = os.path.join(in_directory, name)
        im = Image.open(infile)
        im = im.convert(mode="RGB")
        # print("Generating new format for {} to new file {}".format(name, out_name))
        im.save(outfile)
    except OSError:
        # print("Skipping invalid file {}".format(name))
        return


def convert_scene_to_img_and_voxelmap(in_directory, out_directory, scene_id):
    if 'pbar' in globals() and 'counter' in globals():
        global counter
        counter += 1
        pbar.update(counter)

    cameras = load_scene_db_data(scene_id)
    image_name = get_main_image_name(cameras)

    convert_tiff(in_directory, out_directory, image_name, image_name+'.tiff')

    outfile = os.path.join(out_directory, "{}.rick".format(image_name))
    if os.path.exists(outfile):
        return
    
    voxelmap, image_name = scene_to_voxelmap(cameras)
    with open('voxelmap-{}.rick'.format(image_name), 'wb+') as f:
        pickle.dump(voxelmap, f)


### actually running the extraction

In [ ]:
workers = 10
run_id = 6

widgets = [progressbar.Percentage(), ' ', progressbar.Counter(), ' ', progressbar.Bar(), ' ',
           progressbar.FileTransferSpeed()]

pbar = progressbar.ProgressBar(widgets=widgets, maxval=len(scenes)).start()
counter = 0

Parallel(n_jobs=workers, backend='threading')(delayed(convert_scene_to_img_and_voxelmap)(in_directory, out_directory, i['scene_id']) for i in scenes)


### running extraction for one scene

In [8]:
convert_scene_to_img_and_voxelmap(in_directory, out_directory, scenes[0]['scene_id'])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [9]:
np.array_equal([1, 0], [0, 0, 0])

False